In [2]:
import pandas as pd
pd.set_option("display.max_seq_items", None)  # <- enlève le "..."
pd.set_option("display.width", None)          # optionnel

In [3]:
raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")

C:\Users\alexi\AppData\Local\Temp\ipykernel_28148\2691490454.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")


In [4]:
data = raw_data.copy()

In [5]:
# for 2005 to 2025, read pickle file get gvkeys into a dictionary and delete the dataframe to free up memory
gvkeys_dict = {}
for year in range(2005, 2026):
    text_data = pd.read_pickle(f"C:\\Users\\alexi\\OneDrive\\Documents\\école\\McGill-FIAM\\2025\\Hackathon-Final-2025\\DATA ASSET MANAGEMENT HACKATHON 2025 FINALS\\TEXT DATA US by YEAR\\{year}\\text_us_{year}.pkl")
    gvkeys_dict[year] = text_data['gvkey'].unique().tolist()
    del text_data

In [6]:
# For every gvkey in the dictionary, filter the data dataframe to only include rows with those gvkeys for the corresponding year
filtered_data_list = []
for year in range(2005, 2026):
    # data from that year contains only gvkeys in gvkeys_dict[year]
    filtered_data = data[(data['year'] == year) & (data['gvkey'].isin(gvkeys_dict[year]))]
    filtered_data_list.append(filtered_data)

# concatenate all filtered dataframes
filtered_data = pd.concat(filtered_data_list, ignore_index=True)
del filtered_data_list

In [7]:
# Keep columns: date, excntry, stock_ret, year, month, char_date, market_equity, be_me, ni_me, at_gr1, tangibility, at_be, debt_me, div12m_me, eqpo_me, eqnetis_at, debt_iss, ni_be, profit_sale, gp_at, turnover_126d
filtered_data = filtered_data[['date', 'gvkey', 'excntry', 'stock_ret', 'year', 'month', 'char_date', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d']]
filtered_data.head()

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,at_be,debt_me,div12m_me,eqpo_me,eqnetis_at,dbnetis_at,ni_be,ebit_sale,gp_at,turnover_126d
0,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,2.687348,0.615839,0.014944,0.014924,0.002440,-0.040043,0.060446,0.064513,0.132266,0.002816
1,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,2.380715,0.359312,0.007424,0.102224,-0.053957,0.112408,0.122037,0.214775,0.259079,0.003992
2,20050228,3153.0,CAN,0.135651,2005,2,20050131,844.238381,0.204080,-0.045023,...,2.393489,0.239259,NaN,0.000939,0.030358,-0.082417,-0.220614,-0.113432,0.102018,NaN
3,20050228,4040.0,CAN,0.004546,2005,2,20050131,7294.809288,0.595438,0.030264,...,1.951970,0.243202,0.028309,0.023708,0.010318,-0.134135,0.050826,0.073692,0.191872,0.000004
4,20050228,4864.0,USA,0.207941,2005,2,20050131,602.468980,NaN,-0.449718,...,NaN,0.957732,NaN,0.009199,0.000000,-0.067308,NaN,0.013590,0.118811,0.038071


In [8]:
joining_table = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/North America Company Name Merge by DataDate-GVKEY-IID.csv")
# rename datadate to date
joining_table = joining_table.rename(columns={"datadate": "date"})
joining_table

,gvkey,date,iid,tic,cusip,conm,cik
0,1003,2005-01-31,01,ANTQ,000354100,A.A. IMPORTING CO INC,730052.0
1,1004,2005-01-31,01,AIR,000361105,AAR CORP,1750.0
2,1009,2005-01-31,01,ABSI.1,000781104,ABS INDUSTRIES INC,313368.0
3,1013,2005-01-31,01,ADCT.1,000886309,ADC TELECOMMUNICATIONS INC,61478.0
4,1019,2005-01-31,01,AFAP,001038108,AFA PROTECTIVE SYSTEMS INC,2668.0
...,...,...,...,...,...,...,...
4438939,364383,2025-06-30,01,ITGLF,45829L107,INTEGRAL METALS CORP,2021206.0
4438940,364659,2025-06-30,01,AAGAF,827719105,SILVER47 EXPLORATION CORP,2079414.0
4438941,364659,2025-06-30,01C,AGA.,827719105,SILVER47 EXPLORATION CORP,2079414.0
4438942,364873,2025-06-30,01,TUNGF,030338107,AMERICAN TUNGSTEN CORP,2049560.0


In [ ]:
# --- Keys côté filtered_data ---
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['char_date_key'] = pd.to_datetime(filtered_data['char_date'].astype(str), format='%Y%m%d', errors='coerce')

# --- Keys côté joining_table ---
joining_table['gvkey_key'] = pd.to_numeric(joining_table['gvkey'], errors='coerce').astype('Int64')
joining_table['char_date_key'] = pd.to_datetime(joining_table['date'], errors='coerce')

# --- Merge sur gvkey + char_date ---
filtered_data = filtered_data.merge(
    joining_table[['gvkey_key', 'char_date_key', 'tic', 'conm']],
    on=['gvkey_key', 'char_date_key'],
    how='left'
)

In [10]:
filtered_data

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,eqnetis_at,dbnetis_at,ni_be,ebit_sale,gp_at,turnover_126d,gvkey_key,char_date_key,tic,conm
0,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.002440,-0.040043,0.060446,0.064513,0.132266,0.002816,1243,2005-01-31,AL.1,ALCAN INC
1,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.002440,-0.040043,0.060446,0.064513,0.132266,0.002816,1243,2005-01-31,AL.Z,ALCAN INC
2,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,-0.053957,0.112408,0.122037,0.214775,0.259079,0.003992,2697,2005-01-31,NXY,NEXEN INC
3,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,-0.053957,0.112408,0.122037,0.214775,0.259079,0.003992,2697,2005-01-31,NXY.,NEXEN INC
4,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,-0.053957,0.112408,0.122037,0.214775,0.259079,0.003992,2697,2005-01-31,NXY.PB,NEXEN INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657649,20250630,315318.0,USA,0.063407,2025,6,20250530,5184.735520,0.477729,0.046791,...,0.000000,-0.023267,0.097945,0.146526,0.244835,0.010482,315318,2025-05-30,NaN,NaN
657650,20250630,316056.0,USA,0.013758,2025,6,20250530,12279.192300,0.122215,0.048660,...,-0.045746,-0.089353,0.398148,0.209984,0.397656,0.011089,316056,2025-05-30,NaN,NaN
657651,20250630,317264.0,USA,0.138720,2025,6,20250530,913.093680,1.170207,0.176673,...,-0.002144,-0.027848,0.150976,12.130158,-0.067016,0.018638,317264,2025-05-30,NaN,NaN
657652,20250630,326688.0,USA,0.113222,2025,6,20250530,10833.048800,0.321267,0.022228,...,-0.014343,0.055873,0.069189,0.185390,0.201206,0.012945,326688,2025-05-30,NaN,NaN


In [12]:
sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")
sector_mapping

C:\Users\alexi\AppData\Local\Temp\ipykernel_28148\1981980180.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")


,id,date,eom,gvkey,iid,excntry,gics,sic,naics,year,month
0,comp_001004_01,20250131,20250131,1004.0,01,USA,20101010.0,NaN,NaN,2025,1
1,comp_001004_01,20250228,20250228,1004.0,01,USA,20101010.0,NaN,NaN,2025,2
2,comp_001004_01,20250331,20250331,1004.0,01,USA,20101010.0,NaN,NaN,2025,3
3,comp_001004_01,20250430,20250430,1004.0,01,USA,20101010.0,NaN,NaN,2025,4
4,comp_001004_01,20250530,20250531,1004.0,01,USA,20101010.0,NaN,NaN,2025,5
...,...,...,...,...,...,...,...,...,...,...,...
6924137,crsp_93436,20240830,20240831,184996.0,1.0,USA,25102010.0,3711.0,336111.0,2024,8
6924138,crsp_93436,20240930,20240930,184996.0,1.0,USA,25102010.0,3711.0,336111.0,2024,9
6924139,crsp_93436,20241031,20241031,184996.0,1.0,USA,25102010.0,3711.0,336111.0,2024,10
6924140,crsp_93436,20241129,20241130,184996.0,1.0,USA,25102010.0,3711.0,336111.0,2024,11


In [13]:
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['date_key']  = pd.to_datetime(filtered_data['date'].astype(str), format='%Y%m%d', errors='coerce')

# --- Keys sector_mapping (date = YYYYMMDD en int) ---
sm = sector_mapping[['gvkey','date','gics','sic','naics']].copy()
sm['gvkey_key'] = pd.to_numeric(sm['gvkey'], errors='coerce').astype('Int64')
sm['date_key']  = pd.to_datetime(sm['date'].astype(str), format='%Y%m%d', errors='coerce')

# (optionnel) éviter les duplications si plusieurs lignes par gvkey-date
sm = sm.drop_duplicates(subset=['gvkey_key','date_key'])

# --- Merge ---
filtered_data = filtered_data.merge(
    sm[['gvkey_key','date_key','gics','sic','naics']],
    on=['gvkey_key','date_key'],
    how='left'
).drop(columns=['gvkey_key','date_key'])

In [24]:
# 1) Extraire le code secteur GICS (2 premiers chiffres) à partir du code GICS 8 chiffres
# ex: 20101010 -> 20
filtered_data['gics_sector_code'] = (
    pd.to_numeric(filtered_data['gics'], errors='coerce')
      .floordiv(10**6)
      .astype('Int64')
)

# 2) Mapping MSCI / GICS (11 secteurs)
gics_sector_map = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",
    25: "Consumer Discretionary",
    30: "Consumer Staples",
    35: "Health Care",
    40: "Financials",
    45: "Information Technology",
    50: "Communication Services",
    55: "Utilities",
    60: "Real Estate",
}

# 3) Ajouter le nom du secteur
filtered_data['gics_sector_name'] = filtered_data['gics_sector_code'].map(gics_sector_map)
filtered_data

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,gp_at,turnover_126d,char_date_key,tic,conm,gics,sic,naics,gics_sector_code,gics_sector_name
0,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.132266,0.002816,2005-01-31,AL.1,ALCAN INC,15104010.0,3350.0,331319.0,15,Materials
1,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.132266,0.002816,2005-01-31,AL.Z,ALCAN INC,15104010.0,3350.0,331319.0,15,Materials
2,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
3,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY.,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
4,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY.PB,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657649,20250630,315318.0,USA,0.063407,2025,6,20250530,5184.735520,0.477729,0.046791,...,0.244835,0.010482,2025-05-30,NaN,NaN,15101050.0,NaN,NaN,15,Materials
657650,20250630,316056.0,USA,0.013758,2025,6,20250530,12279.192300,0.122215,0.048660,...,0.397656,0.011089,2025-05-30,NaN,NaN,20102010.0,NaN,NaN,20,Industrials
657651,20250630,317264.0,USA,0.138720,2025,6,20250530,913.093680,1.170207,0.176673,...,-0.067016,0.018638,2025-05-30,NaN,NaN,10102040.0,NaN,NaN,10,Energy
657652,20250630,326688.0,USA,0.113222,2025,6,20250530,10833.048800,0.321267,0.022228,...,0.201206,0.012945,2025-05-30,NaN,NaN,20104010.0,NaN,NaN,20,Industrials


In [30]:
filtered_data['tic'].unique()

array(['AL.1', 'AL.Z', 'NXY', ..., 'MRP', 'UMH.PD', 'FLYYQ'],
      shape=(6070,), dtype=object)

In [31]:
from pathlib import Path
import pandas as pd
import ast
import re

# Répertoire (relatif au notebook)
SP500_DIR = Path("sp500-master/sp500_constituants_2005_2024")

def norm_tic(s):
    if pd.isna(s):
        return pd.NA
    s = str(s).strip().upper()
    # optionnel: harmoniser BRK.B vs BRK-B, BF.B vs BF-B, etc.
    s = s.replace("-", ".")
    return s

# 1) Charger tous les fichiers annuels et construire (year, tic_norm)
members = []
for fp in sorted(SP500_DIR.glob("*-sp500-ticker-list.csv")):
    # année depuis le nom du fichier (ex: 2006-sp500-ticker-list.csv)
    year = int(fp.name.split("-")[0])

    df = pd.read_csv(fp)
    # tickers est une string du type "['A', 'AAPL', ...]"
    tickers = ast.literal_eval(df.loc[0, "tickers"]) if isinstance(df.loc[0, "tickers"], str) else df.loc[0, "tickers"]
    tickers = [norm_tic(t) for t in tickers]

    members.append(pd.DataFrame({"year": year, "tic_norm": tickers}))

sp500_members = pd.concat(members, ignore_index=True).dropna().drop_duplicates()

# 2) Filtrer filtered_data par année (S&P500 de l'année correspondante)
fd = filtered_data.copy()
fd["tic_norm"] = fd["tic"].map(norm_tic)

filtered_data = (
    fd.merge(sp500_members, on=["year", "tic_norm"], how="inner")
      .drop(columns=["tic_norm"])
)


In [34]:
filtered_data.to_csv("yfinance/filtered_sp500_data.csv", index=False)

In [36]:
filtered_data.columns

Index(['date', 'gvkey', 'excntry', 'stock_ret', 'year', 'month', 'char_date',
       'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be',
       'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be',
       'ebit_sale', 'gp_at', 'turnover_126d', 'char_date_key', 'tic', 'conm',
       'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name'],
      dtype='object')